#### Competitor Price

Мы занимаемся динамическим ценообразованием и хотим, чтобы наша модель учитывала цены конкурентов. Пусть для каждого товара (sku) у нас имеется base_price – наша текущая цена. На каждый наш товар благодаря команде матчинга найдены 1, несколько или ни одной цен конкурентов – они записаны в колонке comp_price. Конкуренты имеют приоритет (колонка rank), причём в рангах могут быть пропуски (либо -1, если не найдено ни одной цены конкурента).

Поскольку может быть найдено несколько цен конкурентов, их необходимо согласно какому-то правилу агрегировать. В колонке agg указан тип агрегации:

1.	'avg' – берем среднее

2.	'med' – медиану

3.	'min' – минимальную цену

4.	'max' – максимальную

5.	'rnk' – цену конкурента, имеющего наибольший приоритет (наименьший ранг)

После агрегации мы записываем агрегированную цену конкурента в колонку comp_price.

Осталось определиться, какую брать финальную цену new_price:

•	если для товара цен конкурентов не найдено, оставляем старую цену

•	если агрегированная цена конкурента отличается не более чем на ± 20% от старой цены, ставим её, иначе оставляем старую

________________________________________
> Описание решения

Напишите функцию agg_comp_price, которая на вход принимает датафрейм, группирует его по полю sku, вариант группировки указан в поле agg (Для одинаковых sku, agg одинаковые).
Решение должно быть отсортировано по sku, а индекс начинаться с 0 и быть последовательным.


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

def agg_comp_price(X: pd.DataFrame) -> pd.DataFrame:
    aggs = {'avg':'mean', 'med':'median', 'min':lambda x: min(x), 'max':lambda x: max(x), 'rnk':None}
    skus = X.sku.unique()
    agg_price = []
    for sku in skus:
        agg_function = X[X['sku']==sku]['agg'].values[0]
        if agg_function != 'rnk':
            new = X[X['sku']==sku].groupby(['sku', 'agg'], as_index=False).agg(aggs[agg_function])['comp_price'].values[0]
            agg_price.append(new)
        else:
            rank = min(X[X['sku']==sku]['rank'].values)
            new = X[(X['sku']==sku)&(X['rank']==rank)]['comp_price'].values[0]
            agg_price.append(new)
    X = X.groupby(['sku', 'agg', 'base_price'], as_index=False).count()
    X.drop('rank', axis=1, inplace=True)
    X['comp_price']=agg_price
    X['new_price']=np.where(np.abs(1-(X['base_price']/X['comp_price'])) > 0.2, 
                                 X['base_price'], X['comp_price'])
    X['new_price']=np.where(np.isnan(X['new_price']), X['base_price'], X['new_price'])
    return X

In [3]:
df = pd.read_csv('comp_price.csv', sep=';')
df.head()

,sku,agg,rank,base_price,comp_price
0,0,max,-1,33.0,NaN
1,1,med,0,17.7,16.4
2,1,med,1,17.7,21.8
3,2,avg,0,76.7,77.0
4,2,avg,1,76.7,73.9


In [4]:
agg_comp_price(df)

,sku,agg,base_price,comp_price,new_price
0,0,max,33.0,NaN,33.00
1,1,med,17.7,19.10,19.10
2,2,avg,76.7,75.45,75.45
3,3,rnk,39.7,37.40,37.40
4,4,max,18.0,22.40,22.40
5,5,max,84.8,106.00,84.80
6,6,min,73.6,31.70,73.60
7,7,med,58.6,71.30,71.30
8,8,rnk,35.2,NaN,35.20
9,9,rnk,87.0,88.20,88.20


#### Elasticity

Вам необходимо дописать функцию elasticity_df, которая принимает на вход датасет и возвращает эластичность для каждого SKU.
________________________________________
Чтобы подсчитать эластичность:

1.	Для каждого товара постройте линейную зависимость логарифма продаж от цены.

2.	Возьмите коэффициент детерминации линейной регрессии R2 как оценку эластичности для данного товара


In [5]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf


def elasticity_df(df: pd.DataFrame) -> pd.DataFrame:


    df.sort_values('sku', inplace = True)
    skus = df.sku.unique()
    rsquared = []
    for sku in skus:
        df_part = df[df['sku']==sku]
        model = smf.ols(formula = 'np.log(qty + 1) ~ price', data = df_part)
        result = model.fit()
        if result.rsquared == -np.inf:
            rsquared.append(0)
        else:
            rsquared.append(result.rsquared)
    new_df = df.groupby('sku', as_index = False).count()
    new_df.drop(['dates', 'price', 'qty'], axis = 1, inplace = True)
    new_df['elasticity'] = rsquared             
    return new_df

In [6]:
elasticity_df(df).head()

PatsyError: Error evaluating factor: NameError: name 'qty' is not defined
    np.log(qty + 1) ~ price
    ^^^^^^^^^^^^^^^

#### Similar Items Price

В данной задаче мы сконцентрируемся на ценах на похожих товаров. 

Вам даны два словаря: 

1.	embeddings - словарь с эмбеддингами, в качестве ключей выступают идентификаторы товаров. Гарантируется, что все вектора одной длины.

2.	prices - словарь с ценами, в качестве ключей выступают идентификаторы товаров. В нем нет пустых значений.

Вам нужно заполнить класс SimilarItems с 3 вспомогательными методами и основную функцию transform - она возвращает новую цену для каждого товара.

Функция distances
________________________________________
Функция считает попарные расстояния между всеми эмбеддингами, возвращая словарь расстояний.

Функция knn
________________________________________
На вход функция принимает результат работы функции distance, и параметр top - кол-во ближайших соседей. Она выдает словарь с парами item_id - список top ближайших товаров.

Функция knn_price
________________________________________
На вход функция принимает результат работы функции knn и словарь price с ценами для каждого товара. На выходе выдавая средневзвешенную цену top ближайших соседей. Округлите новую цену до двух знаков после запятой.

Функция transform
________________________________________
Преобразует исходный словарь эмбеддингов в словарь с новыми ценами для всех товаров.


In [ ]:
from pydantic import BaseModel
from typing import Dict, Tuple, List
import numpy as np
from numpy.linalg import norm
from itertools import combinations

class SimilarItems(BaseModel):

    @staticmethod
    def distances(
        embeddings: Dict[int, np.ndarray]
    ) -> Dict[Tuple[int, int], float]:
        """Calculate pairwise distances between each item
        in embedding. We use cosine metric.

        Args:
            embeddings (Dict[int, np.ndarray]): Items embeddings

        Returns:
            Dict[Tuple[int, int], float]: Pairwise distances dict
            Keys are in form of (i, j) - pairs of item_ids stored
            in compact way
        """
        pair_dists = {}
        keys = combinations(embeddings.keys(), r=2)
        for comb in keys:
            A = embeddings[comb[0]]
            B = embeddings[comb[1]]
            cosine = np.dot(A,B)/(norm(A)*norm(B))
            pair_dists[comb] = cosine            
        return pair_dists
    
    @staticmethod
    def knn(
        dist: Dict[Tuple[int, int], float], top: int
    ) -> Dict[int, List[Tuple[int, float]]]:
        """Return closest neighbors for each item.

        Args:
            dist (Dict[Tuple[int, int], float]): <distances> method output
            top (int): Number of top neighbors to consider.

        Returns:
            Dict[int, List[Tuple[int, float]]]: Dict with top closer neighbors
            for each item.
        """
        dist_dict = {}

        for key, value in dist.items():
            dist_dict.setdefault(key[0], [])    
            dist_dict[key[0]].append((key[1], value))
            dist_dict.setdefault(key[1], [])
            val = (key[0], value)
            if val not in dist_dict[key[1]]:
                dist_dict[key[1]].append(val)
                
        knn_dict = {}
        for key, value in dist_dict.items():
            knn_dict[key] = sorted(value, key=lambda x: x[1], reverse=True)[:top]
        return knn_dict    
    
    @staticmethod
    def knn_price(
        knn_dict: Dict[int, List[Tuple[int, float]]],
        prices: Dict[int, float],
    ) -> Dict[int, float]:
        """Calculate weighted average prices for each item.

        Args:
            knn_dict (Dict[int, List[Tuple[int, float]]]): <knn> method output.
            prices (Dict[int, float]): Price dict for each item.

        Returns:
            Dict[int, float]: New prices dict, rounded to 2 decimal places.
        """
        knn_price_dict = {}

        for key, value in knn_dict.items():
            price = 0
            weight = 0
            for item in value:
                price += prices[item[0]]*item[1]
                weight += item[1]
            price /= weight
            knn_price_dict[key]=round(price, 2)
            
        return knn_price_dict 
    
    @staticmethod
    def transform(
        embeddings: Dict[int, np.ndarray],
        prices: Dict[int, float],
        top: int,
    ) -> Dict[int, float]:
        """Transforming input embeddings into a dictionary
        with weighted average prices for each item.

        Args:
            embeddings (Dict[int, np.ndarray]): Items embeddings
            prices (Dict[int, float]): Price dict for each item
            top (int): Number of top neighbors to consider

        Returns:
            Dict[int, float]: Dict with weighted average prices for each item
        """
        dist = SimilarItems.distances(embeddings)
        knn_dict = SimilarItems.knn(dist, top)
        knn_price_dict = SimilarItems.knn_price(knn_dict, prices)
        return knn_price_dict

In [ ]:
embeddings = {
    1: np.array([-26.57, -76.61, 81.61, -9.11, 74.8, 54.23, 32.56, -22.62, -72.44, -82.78]),
    2: np.array([-55.98, 82.87, 86.07, 18.71, -18.66, -46.74, -68.18, 60.29, 98.92, -78.95]),
    3: np.array([-27.97, 25.39, -96.85, 3.51, 95.57, -27.48, -80.27, 8.39, 89.96, -36.68]),
    4: np.array([-37.0, -49.39, 43.3, 73.36, 29.98, -56.44, -15.91, -56.46, 24.54, 12.43]),
    5: np.array([-22.71, 4.47, -65.42, 10.11, 98.34, 17.96, -10.77, 2.5, -26.55, 69.16]),
}


prices = {
    1: 100.5,
    2: 12.2,
    3: 60.0,
    4: 11.1,
    5: 245.2
}

In [ ]:
sim = SimilarItems()
sim.transform(embeddings, prices, 3)

#### nDCG

In [ ]:
# Cumulative Gaine

from typing import List

import numpy as np


def cumulative_gain(relevance: List[float], k: int) -> float:
    """Score is cumulative gain at k (CG@k)

    Parameters
    ----------
    relevance:  `List[float]`
        Relevance labels (Ranks)
    k : `int`
        Number of elements to be counted

    Returns
    -------
    score : float
    """
    #top_relevance = sorted(relevance, reverse=True)
    score = np.sum(relevance[:k])
    return score

In [ ]:
relevance = [0.99, 0.94, 0.88, 0.74, 0.71, 0.68]
k = 5
print(cumulative_gain(relevance, k))

In [ ]:
from typing import List

import numpy as np


def discounted_cumulative_gain(relevance: List[float], k: int, method: str = "standard") -> float:
    """Discounted Cumulative Gain

    Parameters
    ----------
    relevance : `List[float]`
        Video relevance list
    k : `int`
        Count relevance to compute
    method : `str`, optional
        Metric implementation method, takes the values \
            `standard` and `industry`

    Returns
    -------
    score : `float`
        Metric score
    """    
    score = 0
    if method == "industry":
        for i, v in enumerate(relevance[:k], 1):
            score += (2*v-1)/np.log2(i+1)
            return score
    for i, v in enumerate(relevance[:k], 1):
        score += v/np.log2(i+1)
    return score

In [ ]:
relevance = [0.99, 0.94, 0.88, 0.74, 0.71, 0.68]
k = 5
method = 'standard'
print(discounted_cumulative_gain(relevance, k, method))

In [ ]:
from typing import List

import numpy as np


def normalized_dcg(relevance: List[float], k: int, method: str = "standard") -> float:
    """Normalized Discounted Cumulative Gain.

    Parameters
    ----------
    relevance : `List[float]`
        Video relevance list
    k : `int`
        Count relevance to compute
    method : `str`, optional
        Metric implementation method,
        takes the values `standard` and `industry`

    Returns
    -------
    score : `float`
        Metric score
    """
    dcg = 0
    idcg = 0
    if method == "industry":
        for i, v in enumerate(relevance[:k], 1):
            dcg += (2*v-1)/np.log2(i+1)
            
        for i, v in enumerate(sorted(relevance[:k], reverse = True), 1):
            idcg += (2*v-1)/np.log2(i+1)

    else:   
        for i, v in enumerate(relevance[:k], 1):
            dcg += v/np.log2(i+1)  

        for i, v in enumerate(sorted(relevance[:k], reverse = True), 1):
            idcg += v/np.log2(i+1)
    score = dcg/idcg
    return score

In [ ]:
relevance = [0.99, 0.94, 0.74, 0.88, 0.71, 0.68]
k = 5 
method = 'standard'
print(normalized_dcg(relevance, k, method))

In [ ]:
from typing import List

import numpy as np

def avg_ndcg(list_relevances: List[List[float]], k: int, method: str = 'standard') -> float:
    """avarage nDCG

    Parameters
    ----------
    list_relevances : `List[List[float]]`
        Video relevance matrix for various queries
    k : `int`
        Count relevance to compute
    method : `str`, optional
        Metric implementation method, takes the values\
             `standard` and `industry`

    Returns
    -------
    score : `float`
        Metric score
    """

    score = np.mean(list(map(lambda x: (normalized_dcg(x, k, method)), list_relevances)))
    return score

list_relevances = [
        [0.99, 0.94, 0.88, 0.89, 0.72, 0.65],
        [0.99, 0.92, 0.93, 0.74, 0.61, 0.68], 
        [0.99, 0.96, 0.81, 0.73, 0.76, 0.69]
    ]  
k = 5
method = 'standard'
print(round(avg_ndcg(list_relevances, k, method), 5))

#### Joblib

In [ ]:
import re
from string import punctuation

import pandas as pd
from joblib import delayed
from joblib import Parallel
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


def text_processing(text):
    text = str(text)
    text = re.sub(r"https?://[^,\s]+,?", "", text)
    text = re.sub(r"@[^,\s]+,?", "", text)

    stop_words = stopwords.words("english")
    transform_text = text.translate(str.maketrans("", "", punctuation))
    transform_text = re.sub(" +", " ", transform_text)

    text_tokens = word_tokenize(transform_text)

    lemma_text = [
        lemmatizer.lemmatize(word.lower()) for word in text_tokens
        ]
    
    cleaned_text = " ".join(
            [str(word) for word in lemma_text if word not in stop_words]
        )
    return cleaned_text
    

def clear_data(source_path: str, target_path: str, n_jobs: int):
    """Parallel process dataframe

    Parameters
    ----------
    source_path : str
        Path to load dataframe from

    target_path : str
        Path to save dataframe to

    n_jobs : int
        Count of job to process
    """
    
    data = pd.read_parquet(source_path)
    data = data.copy().dropna().reset_index(drop=True)
    lemmatizer = WordNetLemmatizer()

    cleaned_text_list = Parallel(
        n_jobs=n_jobs, backend="multiprocessing", verbose=5 * n_jobs
    )(delayed(text_processing)(text) for text in data["text"])
    data["cleaned_text"] = cleaned_text_list
    data.to_parquet(target_path)